 Implement k-means algorithm for the given data set for different values of k.  
 Evaluate your algorithm using rand index with respect to weather_main attribute as ground truth cluster label.

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, random, math
from collections import Counter

from scipy import stats

Reading from CSV file

In [2]:
df = pd.read_csv("DataSetk-means.csv")

Reading from ods file

In [3]:
df_ods = pd.read_excel("DataSetk-means.ods", engine="odf")

In [4]:
df

,temp,rain_1h,snow_1h,clouds_all,weather_description,traffic_volume,weather_main
0,288.28,0,0,40,scattered clouds,5545,Clouds
1,289.36,0,0,75,broken clouds,4516,Clouds
2,289.58,0,0,90,overcast clouds,4767,Clouds
3,290.13,0,0,90,overcast clouds,5026,Clouds
4,291.14,0,0,75,broken clouds,4918,Clouds
...,...,...,...,...,...,...,...
494,280.57,0,0,40,scattered clouds,259,Clouds
495,280.54,0,0,75,broken clouds,324,Clouds
496,279.37,0,0,8,mist,806,Mist
497,279.18,0,0,8,mist,2622,Mist


In [5]:
len(Counter(df["weather_description"]))

14

In [6]:
Counter(df["weather_description"])

Counter({'scattered clouds': 31,
         'broken clouds': 66,
         'overcast clouds': 110,
         'sky is clear': 107,
         'few clouds': 42,
         'light rain': 34,
         'light intensity drizzle': 24,
         'mist': 58,
         'haze': 1,
         'fog': 9,
         'proximity shower rain': 1,
         'drizzle': 10,
         'moderate rain': 5,
         'heavy intensity rain': 1})

In [7]:
Counter(df["rain_1h"])

Counter({0: 499})

In [8]:
Counter(df["snow_1h"])

Counter({0: 499})

2 columns "rain_1h" and "snow_1h" both are 0 and can be removed

In [9]:
print("Removing rain_1h and snow_1h")
df.drop(["rain_1h", "snow_1h"], inplace=True, axis=1)

Removing rain_1h and snow_1h


In [10]:
df.columns

Index(['temp', 'clouds_all', 'weather_description', 'traffic_volume',
       'weather_main'],
      dtype='object')

In [11]:
for column in df.columns:
    print(f"{column}: {df[column].dtype}")

temp: float64
clouds_all: int64
weather_description: object
traffic_volume: int64
weather_main: object


## Normalising the data

In [12]:
for column in df.columns:
    if df[column].dtype != "object":
        mean = df[column].mean()
        std = df[column].std()
        df[column] = (df[column]-mean)/std

In [13]:
df

,temp,clouds_all,weather_description,traffic_volume,weather_main
0,1.244246,-0.571764,scattered clouds,1.051308,Clouds
1,1.461489,0.375930,broken clouds,0.543897,Clouds
2,1.505742,0.782085,overcast clouds,0.667668,Clouds
3,1.616375,0.782085,overcast clouds,0.795384,Clouds
4,1.819538,0.375930,broken clouds,0.742128,Clouds
...,...,...,...,...,...
494,-0.306628,-0.571764,scattered clouds,-1.555278,Clouds
495,-0.312663,0.375930,broken clouds,-1.523226,Clouds
496,-0.548009,-1.438227,mist,-1.285546,Mist
497,-0.586228,-1.438227,mist,-0.390056,Mist


In [14]:
X = df.drop("weather_main", axis=1)
y = df.weather_main

In [15]:
Counter(y)

Counter({'Clouds': 249,
         'Clear': 107,
         'Rain': 41,
         'Drizzle': 34,
         'Mist': 58,
         'Haze': 1,
         'Fog': 9})

In [16]:
print(f"Total number of classes: {len(Counter(y))}")

Total number of classes: 7


In [91]:
# Converting pandas dataframe x to numpy array
X = X.to_numpy()

## K means clustering

Algo:
* Initialize clusters centroids $\mu_1, \mu_2, \cdots, \mu_k$
* Repeat until convergence: {  
    For every i, set  
 $c^{(i)} = arg min ||x^{(i)} - \mu_j ||^2$   
    For each j, set  
$\mu_j = \dfrac{\sum_{i=1}^{m} 1{\{c^{(i)} = j\}x^{(i)}}}{\sum_{i=1}^{m} 1\{c^{(i)} = j\} }$
    
 
}

In [ ]:
def dist(a, b):
    '''
    Calculate distance between two different rows/objects
    a: numpy array
    b: numpy array
    '''
    ans = 0
    for i in range(len(a)):
        if isinstance(a[i], str):
            if a[i] != b[i]:
                ans += 1
        else:
            ans += (a[i] - b[i])**2
    return math.sqrt(ans)

def get_centroid(X, mp):
    '''
    Calculating centroid of a given set of objects
    mp: a dict or map which stores indices of values having same cluster
    returns: cetroid of cluster passed
    '''
    centroid = []
    for cluster_index in mp:
        curr_centroid = []
        for i in range(X.shape[1]):
            # iterating for every features
            if isinstance(X[0][i], str):
                # calculating mode
                curr_centroid.append(stats.mode(X[mp[cluster_index], i])[0][0])
            else:
                try:
                    curr_centroid.append(X[mp[cluster_index],i].mean())
                except:
                    print("ERROR OCCURED")
                    print(f"cluster_index is {cluster_index}")
                    print(mp)
        centroid.append(curr_centroid)
    return centroid
    
def initialise_dict(mp, n_classes):
    '''
    Takes a dict and clear it. Initlise that dict with n_classes empty list
    '''
    mp.clear()
    for i in range(n_classes):
        mp[i] = []
    

def kMeans(X, k, max_iter=10000):
    '''
    X: numpy array, training data
    k: number of clusters
    max_iter: maximum number of iterations
    Returns: a map of clusters and final k means
    '''
    N_TOT = X.shape[0]
    # randomly chosing initial centroids from given data
    random_values = random.sample(range(0, N_TOT), k)
    means = [X[i] for i in random_values]
    mp = dict()
    for n_iter in range(max_iter):
        initialise_dict(mp, k)
        for i in range(N_TOT):
            min_dist = int(1e9)
            curr_class = 0
            for j in range(len(means)):
                x = dist(means[j], X[i])
                if x < min_dist:
                    min_dist = x
                    curr_class = j
            mp[curr_class].append(i)
        updated_means = get_centroid(X, mp)
        # compare two means, if they are equal break from loop
        equal = True
        
        for i in range(len(means)):
            for j in range(len(means[i])):
                if isinstance(means[i][j], str):
                    if means[i][j] != updated_means[i][j]:
                        equal = False
                        break
                elif abs(means[i][j] - updated_means[i][j]) > 1e-6:
                    equal = False
                    break
        if equal:
            break
    return mp, means
        
                    
            

In [177]:
mp, means = kMeans(X, 3, 100000)

In [178]:
for i in mp:
    print (len(mp[i]))

266
123
110


## Rand index

$R = \dfrac{a + b}{a + b + c + d} = \dfrac{a + b}{n\choose2}$    
a: number of elements which are in same subsets in both X and Y  
b: number of elements which are in different subsets in both X and Y  


In [191]:
def rand_index(X, y, mp, clusters):
    '''
    X: training data
    y: given class/ground truth
    mp and clusters are the output of kMeans functions, mp is a map  and clusters are centroid of clusters
    '''
    # Creating a few dicts which will help in comparing which subsets a particular row from training example is
#     row_to_index = dict()

#     for i in range(len(X)):
#         row_to_index[X[i]] = i
        
    k_means_cluster_index = dict()
    for i in mp:
        for j in mp[i]:
            k_means_cluster_index[j] = i
    a = 0
    b = 0
    for i in range(len(X)):
        for j in range(i+1, len(X)):
            if (k_means_cluster_index[i] == k_means_cluster_index[j]) and (y[i] == y[j]):
                a += 1
            elif(k_means_cluster_index[i] != k_means_cluster_index[j]) and (y[i] != y[j]):
                b += 1
    n = len(X)
    nc2 = (n*(n-1))/2
    return (a + b)/nc2

In [193]:
mp, means = kMeans(X, 3, 100000)

In [194]:
rand_index(X, y, mp, means)

0.574329381654876

In [197]:
mp, means = kMeans(X, 4, 50000)

In [198]:
rand_index(X, y, mp, means)

0.5958181423087138

In [199]:
mp, means = kMeans(X, 5, 5000)

In [200]:
rand_index(X, y, mp, means)

0.6106027315675527

In [202]:
mp, means = kMeans(X, 6, 5000)
rand_index(X, y, mp, means)

0.607415634481815

In [203]:
mp, means = kMeans(X, 7, 5000)
rand_index(X, y, mp, means)

0.6485501122727383

In [204]:
mp, means = kMeans(X, 8, 5000)
rand_index(X, y, mp, means)

0.6286227072619134

In [205]:
print(mp)

{0: [9, 10, 29, 30, 31, 54, 55, 56, 57, 58, 59, 60, 61, 168, 319, 368, 369, 370, 371, 372], 1: [5, 6, 7, 11, 12, 13, 14, 15, 24, 25, 26, 32, 33, 34, 35, 45, 46, 47, 315, 316, 317, 318, 338, 362, 363, 364, 365, 366, 367], 2: [21, 22, 23, 40, 41, 42, 43, 44, 90, 91, 133, 134, 137, 138, 141, 142, 155, 185, 201, 202, 203, 204, 205, 234, 247, 248, 249, 250, 251, 252, 253, 254, 256, 331, 332, 333, 334, 335, 336, 498], 3: [63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 93, 94, 95, 96, 97, 98, 99, 113, 114, 115, 116, 117, 118, 119, 120, 156, 157, 158, 159, 160, 163, 164, 165, 177, 178, 179, 180, 181, 182, 183, 184, 186, 187, 188, 189, 190, 208, 209, 210, 211, 222, 223, 224, 225, 226, 227, 230, 231, 232, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 307, 308, 309, 310, 311, 312, 313, 343, 355, 358, 359, 381, 382, 383, 384, 386, 387, 388, 389, 390, 391, 392, 393, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 459, 4

In [206]:
mp, means = kMeans(X, 7, 10000)
rand_index(X, y, mp, means)

0.6614675133399329